In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Conv2D,Flatten,Conv2DTranspose
from tensorflow.keras.layers import Reshape, Lambda, Dropout,BatchNormalization,ReLU,LeakyReLU, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.utils import plot_model

#백엔드프로그램 K.int_shape을 쓰면 형태가보임
from tensorflow.keras import backend as K

In [2]:
dir(K)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'abs',
 'all',
 'any',
 'arange',
 'argmax',
 'argmin',
 'backend',
 'batch_dot',
 'batch_flatten',
 'batch_get_value',
 'batch_normalization',
 'batch_set_value',
 'bias_add',
 'binary_crossentropy',
 'binary_focal_crossentropy',
 'cast',
 'cast_to_floatx',
 'categorical_crossentropy',
 'clear_session',
 'clip',
 'concatenate',
 'constant',
 'conv1d',
 'conv2d',
 'conv2d_transpose',
 'conv3d',
 'cos',
 'count_params',
 'ctc_batch_cost',
 'ctc_decode',
 'ctc_label_dense_to_sparse',
 'cumprod',
 'cumsum',
 'depthwise_conv2d',
 'dot',
 'dropout',
 'dtype',
 'elu',
 'epsilon',
 'equal',
 'eval',
 'exp',
 'expand_dims',
 'experimental',
 'eye',
 'flatten',
 'floatx',
 'foldl',
 'foldr',
 'function',
 'gather',
 'get_uid',
 'get_value',
 'gradients',
 'greater',
 'greater_equal',
 'hard_sigmoid',
 'image_data_format',
 'in_test_phase',
 'in_top_k',
 'in_tr

In [3]:
#mse만들때
#K.mean()

In [4]:
import numpy as np
import os
import json
import pickle

In [5]:
class AutoEncoder():
    def __init__(self,
                 input_dim,
                encoder_conv_filters,
                encoder_conv_kernel_size,
                encoder_conv_strides,
                decoder_conv_t_filters,
                decoder_conv_t_kernel_size,
                decoder_conv_t_strides,
                z_dim):
        self.name = 'autoencoder'
        self.input_dim = input_dim
        self.encoder_conv_filters = encoder_conv_filters
        self.encoder_conv_kernel_size = encoder_conv_kernel_size
        self.encoder_conv_strides = encoder_conv_strides
        self.decoder_conv_t_filters = decoder_conv_t_filters
        self.decoder_conv_t_kernel_size = decoder_conv_t_kernel_size
        self.decoder_conv_t_strides = decoder_conv_t_strides
        self.z_dim = z_dim
        
        self.n_layers_encoder = len(encoder_conv_filters)
        self.n_layers_decoder = len(decoder_conv_t_filters)
        
        self._build()
        
        
    def _build(self):
        encoder_input = Input(shape= self.input_dim, name = 'encoder_input')
        x = encoder_input
        
        for i in range(self.n_layers_encoder):
            conv_layer = Conv2D(
                filters = self.encoder_conv_filters[i],
                kernel_size = self.encoder_conv_kernel_size[i],
                strides = self.encoder_conv_strides[i],
                name = 'encoder_conv_'+str(i + 1)
            )
            x = conv_layer(x)
            x = ReLU()(x)
            
        ##
        conv_shape = K.int_shape(x)[1:]
        #conv_shape = (None,12,12,64)
        x = Flatten()(x)
        encoder_output = Dense(self.z_dim,name = 'encoder_output')(x)
        
        self.encoder = Model(inputs = [encoder_input],outputs = [encoder_output])
        
        decoder_input = Input(shape = (self.z_dim),name = 'decoder_input')
        
        #여기가 9216
        x = Dense(np.prod(conv_shape))(decoder_input)
        #여기가  12 12 64
        x = Reshape(conv_shape)(x)
        
        for i in range(self.n_layers_decoder):
            conv_t_layer = Conv2DTranspose(
                filters = self.decoder_conv_t_filters[i],
                kernel_size = self.decoder_conv_t_kernel_size[i],
                strides = self.decoder_conv_t_strides[i],
                name = 'decoder_conv_t_'+str(i+1)
            )
            x = conv_t_layer(x)
            if i < self.n_layers_encoder -1:
                 x = ReLU()(x)
            else:
                 x = Activation('sigmoid')(x)
                    
        decoder_output = x
            
        self.decoder = Model(decoder_input,decoder_output)
        
        model_input = encoder_input
        model_output = self.decoder(encoder_output)
        
        self.model = Model(model_input,model_output)
        
    def compile(self,learning_rate):
        self.learning_rate = learning_rate
        optimizer = Adam(learning_rate = learning_rate)
            
        def loss(t,y):
            return K.mean(K.square(t-y), axis = [1,2,3])
            
        self.model.compile(optimizer = optimizer, loss = 'mse')
            
    def train(self, x_train, batch_size, epochs):
            
            self.model.fit(
                x_train, #자기 자신으로 검증하므로 y_train이 없다
                x_train,
                batch_size = batch_size,
                epochs = epochs
            )

In [6]:
#[]안에있는건 다음 Dense

autoencoder = AutoEncoder(
                input_dim = (28,28,1), 
                encoder_conv_filters = [32,64,64,64],
                encoder_conv_kernel_size = [3,3,3,3],
                encoder_conv_strides= [1,1,1,1],
                decoder_conv_t_filters = [64,64,32,1],
                decoder_conv_t_kernel_size = [3,3,3,3],
                decoder_conv_t_strides = [1,1,1,1],
                z_dim = 2)

In [7]:
autoencoder.model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 26, 26, 32)        320       
                                                                 
 re_lu (ReLU)                (None, 26, 26, 32)        0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 24, 24, 64)        18496     
                                                                 
 re_lu_1 (ReLU)              (None, 24, 24, 64)        0         
                                                                 
 encoder_conv_3 (Conv2D)     (None, 22, 22, 64)        36928     
                                                                 
 re_lu_2 (ReLU)              (None, 22, 22, 64)        0   

In [8]:
import tensorflow
(X_train, _),(_,_) = tensorflow.keras.datasets.mnist.load_data()

In [9]:
X_train.shape

(60000, 28, 28)

In [10]:
autoencoder.compile(1e-4)

In [11]:
autoencoder.train(X_train,32,10)

Epoch 1/10
1875/1875 [==============================] - 287s 153ms/step - loss: 7217.2129
Epoch 2/10
1875/1875 [==============================] - 294s 157ms/step - loss: 7216.9272
Epoch 3/10
1875/1875 [==============================] - 292s 156ms/step - loss: 7216.8911
Epoch 4/10
1875/1875 [==============================] - 298s 159ms/step - loss: 7216.8916
Epoch 5/10
1875/1875 [==============================] - 290s 155ms/step - loss: 7216.8955
Epoch 6/10
1875/1875 [==============================] - 283s 151ms/step - loss: 7216.8789
Epoch 7/10
 408/1875 [=====>........................] - ETA: 3:46 - loss: 7182.5342

KeyboardInterrupt: 

In [12]:
autoencoder.encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_1 (Conv2D)     (None, 26, 26, 32)        320       
                                                                 
 re_lu (ReLU)                (None, 26, 26, 32)        0         
                                                                 
 encoder_conv_2 (Conv2D)     (None, 24, 24, 64)        18496     
                                                                 
 re_lu_1 (ReLU)              (None, 24, 24, 64)        0         
                                                                 
 encoder_conv_3 (Conv2D)     (None, 22, 22, 64)        36928     
                                                                 
 re_lu_2 (ReLU)              (None, 22, 22, 64)        0     

In [13]:
autoencoder.decoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 25600)             76800     
                                                                 
 reshape (Reshape)           (None, 20, 20, 64)        0         
                                                                 
 decoder_conv_t_1 (Conv2DTra  (None, 22, 22, 64)       36928     
 nspose)                                                         
                                                                 
 re_lu_4 (ReLU)              (None, 22, 22, 64)        0         
                                                                 
 decoder_conv_t_2 (Conv2DTra  (None, 24, 24, 64)       36928     
 nspose)                                                   

In [14]:
encode = autoencoder.encoder.predict(np.random.randn(3,28,28,1))
autoencoder.decoder.predict(encode)

1/1 [==============================] - 0s 146ms/step


array([[[[0.42050204],
         [0.33646584],
         [0.2666027 ],
         ...,
         [0.33379152],
         [0.3993076 ],
         [0.45640284]],

        [[0.3740332 ],
         [0.28695896],
         [0.18440665],
         ...,
         [0.18493073],
         [0.28597835],
         [0.40297022]],

        [[0.31466404],
         [0.19056143],
         [0.10594662],
         ...,
         [0.02732319],
         [0.13109228],
         [0.30414966]],

        ...,

        [[0.36718887],
         [0.27013484],
         [0.17808731],
         ...,
         [0.3005183 ],
         [0.3414052 ],
         [0.386159  ]],

        [[0.41426295],
         [0.3366974 ],
         [0.24371274],
         ...,
         [0.3095144 ],
         [0.33088654],
         [0.41402736]],

        [[0.45784   ],
         [0.4245959 ],
         [0.39796242],
         ...,
         [0.35588935],
         [0.39316732],
         [0.44365245]]],


       [[[0.41995507],
         [0.32759002],
         [0.25

In [15]:
x = (np.random.randn(3,28,28,1),autoencoder.encoder.layers[1].get_weights()[0])

In [16]:
x

(array([[[[ 0.29227075],
          [ 0.97175004],
          [ 1.43634347],
          ...,
          [ 1.29108366],
          [-0.89457933],
          [ 1.07026717]],
 
         [[-0.55307883],
          [-0.04672591],
          [-1.30055471],
          ...,
          [-1.99379719],
          [-0.59841046],
          [ 0.67268874]],
 
         [[-2.70932043],
          [-0.4025321 ],
          [-1.02846083],
          ...,
          [ 0.21704254],
          [-0.31298037],
          [ 0.82593215]],
 
         ...,
 
         [[ 0.59330586],
          [-1.23415081],
          [ 0.76720512],
          ...,
          [ 0.6837674 ],
          [-2.73082223],
          [ 0.58805915]],
 
         [[ 0.94382943],
          [-0.5443234 ],
          [ 0.51130047],
          ...,
          [ 2.34618889],
          [ 0.57991147],
          [ 0.60456446]],
 
         [[-0.36985029],
          [-0.14500421],
          [ 0.66942144],
          ...,
          [ 0.07721386],
          [ 0.56418743],
    

In [17]:
K.int_shape(x[0])

(3, 28, 28, 1)